### Part - I: Importing Required Modules/ Packages

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf 
import keras
print(tf.__version__)
print(keras.__version__)
print(pd.__version__)
print(np.__version__)
import pickle

# custom declarations 
from nn_globals import *
from nn_plotting import gaus, fit_gaus, corr_plot
from dataset import muon_data_split
from nn_evaluate import huber_loss, k_fold_validation
from nn_training import lr_schedule
from nn_pruning_module_support import loading_trained_model
from nn_training_pruned_model import (generate_layer_masks, 
                                      create_sparse_model,
                                      train_sparse_model)
from nn_pruning_module_support import __generate_delta_plots__
from nn_pruning_module_support import (saving_pruned_model,loading_pruned_model)
fron nn_evaluate import get_sparsity
from custom_dense_layer import MaskedDense

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (LearningRateScheduler, 
                                        TerminateOnNaN, 
                                        EarlyStopping)
from keras.regularizers import l1_l2
from keras.initializers import glorot_uniform

2.6.0
2.6.0
1.3.4
1.19.5


### Part- II: Data Preprocessing

In [6]:
x_train_displ, x_test_displ, y_train_displ, y_test_displ, \
dxy_train_displ, dxy_test_displ =  muon_data_split(infile_muon_displ, 
                                                    reg_pt_scale=reg_pt_scale, 
                                                    reg_dxy_scale=reg_dxy_scale, 
                                                    test_size=0.315,
                                                    batch_size = 128)

[INFO    ] Loading muon data from /Users/gpradhan/Desktop/optimization-for-cms-muon-trigger-nn/data/NN_input_params_FlatXYZ.npz ...
[INFO    ] Loaded the variables with shape (19300000, 25)
[INFO    ] Loaded the parameters with shape (19300000, 6)
[INFO    ] Loaded the encoded variables with shape (3284620, 23)
[INFO    ] Loaded the encoded parameters with shape (3284620,)
[INFO    ] Loaded # of training and testing events: (2249964, 1034656)
[WARNING ] The last batch for training could be too few! (2024967%128)=7. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*0.9) % 128
[WARNING ] The last batch for training after mixing could be too few! (4049935%128)=15. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*2*0.9) % 128


### Part-III: Use the cells in this section for data exploration

In [ ]:
# cols = ["dphi_1","dphi_2","dphi_3","dphi_4","dphi_5","dphi_6",
#        "dtheta_1","dtheta_2","dtheta_3","dtheta_4","dtheta_5", "dtheta_6",
#        "bend_1","bend_2","bend_3","bend_4",
#        "track theta"]

# x = np.concatenate((x_train_displ,x_test_displ),axis=0)
# y = np.concatenate((y_train_displ,y_test_displ),axis=0)
# dxy = np.concatenate((dxy_train_displ,dxy_test_displ),axis=0)

# corr_plot(x,y,dxy,columns = cols)

### Part- III: Load and Account the Performance for the Baseline

In [ ]:
baseline = loading_trained_model(filepath = "./models/",
                                 model_filename = "model")
baseline.summary()

In [ ]:
# Quantitative Evaluation
k_fold_validation(model = baseline, 
          x = x_test_displ, 
          y = y_test_displ, 
          dxy = dxy_test_displ, 
          folds =1,
          metric_type = "MAE")    
k_fold_validation(model = baseline, 
          x = x_test_displ, 
          y = y_test_displ, 
          dxy = dxy_test_displ, 
          folds =1,
          metric_type = "RMSE")   

# Qualitative Evaluation:
__generate_delta_plots__(model = baseline,
                          x = x_test_displ,
                          y = y_test_displ,
                          dxy = dxy_test_displ,
                          color = "salmon")

### Part-IV: Build the custom model

In [ ]:
def run_iterative_pruning_v1(baseline_model = None,
                          target_sparsity: float = 0.1, 
                          pruning_fraction_step:float = 0.1,
                          training_params: list = []):
    """
    Uses a new train-test-val split for each pruning cycle. 
    """
    
    if target_sparsity > 1.0 or target_sparsity <= 0:
        print("INVALID value entered for target sparsity, it can only be in the range [0,1]")
    if pruning_fraction_step > target_sparsity:
        print("INVALID value entered for pruning fraction, it has to be <= target_sparsity")
    
    # list of models, new pruned models get appended to it while training
    pruned_models, training_history = [],[]
    plot_colors = ["red","orange","blue","cyan","purple","green","magenta", "salmon"]
    init_sparsity = pruning_fraction_step
    i = 0
    while(init_sparsity <= target_sparsity):

        print("-----------------------------------------------------------------------------------------------")
        print("-----------------------------------------------------------------------------------------------")
        print("Currently pruning the model upto {} % of the baseline".format(round(init_sparsity*100)))
        print("-----------------------------------------------------------------------------------------------")
        print("-----------------------------------------------------------------------------------------------")

        x_train_displ, x_test_displ, y_train_displ, y_test_displ, dxy_train_displ, dxy_test_displ =  muon_data_split(infile_muon_displ, 
                                                                                                                       reg_pt_scale=reg_pt_scale, 
                                                                                                                       reg_dxy_scale=reg_dxy_scale, 
                                                                                                                       test_size=0.315)
        y_train_displ = np.abs(y_train_displ)
        y_test_displ = np.abs(y_test_displ)

        # training loop begins
        lr = training_params[i]['lr']
        clipnorm = training_params[i]['clipnorm']
        eps = training_params[i]['eps']
        momentum = training_params[i]['momentum']
        retrain_epochs = training_params[i]['epochs']
        retrain_batch_size = training_params[i]['batch_size']
        l1_reg = training_params[i]['l1_reg']
        l2_reg = training_params[i]['l2_reg']
        sparsity = init_sparsity
        
        # define optimizer, callbacks here
        adam = Adam(lr=lr, clipnorm=clipnorm)
        lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
        terminate_on_nan = TerminateOnNaN()
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=18, 
                                       verbose=True,
                                       mode='min')
        curr_model = None
        if len(pruned_models) == 0:
            curr_model = baseline_model
        else:
            curr_model = pruned_models[-1]

        pruned_model = create_sparse_model(model = curr_model,
                                               input_dim = nvariables,
                                               output_dim = 2,
                                               k_sparsity = sparsity,
                                               bn_epsilon = eps,
                                               bn_momentum = momentum,
                                               l1_reg = l1_reg,
                                               l2_reg = l1_reg,
                                               kernel_initializer="glorot_uniform",
                                               optimizer = adam)

        pruned_model, history = train_sparse_model(sparse_model = pruned_model,
                                                           x = x_train_displ,
                                                           y = y_train_displ,
                                                           dxy = dxy_train_displ,
                                                           retrain_epochs = retrain_epochs,
                                                           batch_size = retrain_batch_size,
                                                           callbacks=[lr_decay, 
                                                                      early_stopping, 
                                                                      terminate_on_nan],
                                                           verbose = True,
                                                           validation_split=0.1)
        
        # evaluate the model
        k_fold_validation(model = pruned_model, 
                  x = x_test_displ, 
                  y = y_test_displ, 
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")    
        k_fold_validation(model = pruned_model, 
                  x = x_test_displ, 
                  y = y_test_displ, 
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "RMSE")   

        __generate_delta_plots__(model = pruned_model,
                                 x = x_test_displ,
                                 y = y_test_displ,
                                 dxy = dxy_test_displ,
                                 color = plot_colors[i])
        
        pruned_models.append(pruned_model)
        training_history.append(history)

        # training ends
        i += 1
        init_sparsity += pruning_fraction_step
    
    return pruned_models, training_history

In [ ]:
def run_iterative_pruning_v2(X_train, y_train, dxy_train,
                            X_test, y_test, dxy_test,
                            baseline_model = None,
                            init_sparsity: float = 0.1,
                            target_sparsity: float = 1.0, 
                            pruning_fraction_step:float = 0.1,
                            training_params: list = [],
                            cv_folds: int = 50):
    """
    Uses the same train-test-val-split for each pruning cycle.
    """
        
    if target_sparsity > 1.0 or target_sparsity <= 0:
        print("INVALID value entered for target sparsity, it can only be in the range [0,1]")
    if pruning_fraction_step > target_sparsity:
        print("INVALID value entered for pruning fraction, it has to be <= target_sparsity")
    
    # list of models, new pruned models get appended to it while training
    pruned_models, training_history = [],[]
    plot_colors = ["red","orange","blue","cyan","purple","navy","teal","salmon"]
    i = 0
    while(init_sparsity <= target_sparsity):

        print("-----------------------------------------------------------------------------------------------")
        print("-----------------------------------------------------------------------------------------------")
        print("Currently pruning the model upto {} % of the baseline".format(round(init_sparsity*100)))
        print("-----------------------------------------------------------------------------------------------")
        print("-----------------------------------------------------------------------------------------------")

        # training loop begins
        lr = training_params[i]['lr']
        clipnorm = training_params[i]['clipnorm']
        eps = training_params[i]['eps']
        momentum = training_params[i]['momentum']
        retrain_epochs = training_params[i]['epochs']
        retrain_batch_size = training_params[i]['batch_size']
        l1_reg = training_params[i]['l1_reg']
        l2_reg = training_params[i]['l2_reg']
        sparsity = init_sparsity
        
        # define optimizer, callbacks here
        adam = Adam(lr=lr, clipnorm=clipnorm)
        lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
        terminate_on_nan = TerminateOnNaN()
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=40, 
                                       verbose=True,
                                       mode='min')
        curr_model = None
        if len(pruned_models) == 0:
            curr_model = baseline_model
        else:
            curr_model = pruned_models[-1]

        pruned_model = create_sparse_model(model = curr_model,
                                               input_dim = nvariables,
                                               output_dim = 2,
                                               k_sparsity = sparsity,
                                               bn_epsilon = eps,
                                               bn_momentum = momentum,
                                               l1_reg = l1_reg,
                                               l2_reg = l1_reg,
                                               kernel_initializer="glorot_uniform",
                                               optimizer = adam)

        pruned_model, history = train_sparse_model(sparse_model = pruned_model,
                                                           x = X_train,
                                                           y = y_train,
                                                           dxy = dxy_train,
                                                           retrain_epochs = retrain_epochs,
                                                           batch_size = retrain_batch_size,
                                                           callbacks=[lr_decay, 
                                                                      early_stopping, 
                                                                      terminate_on_nan],
                                                           verbose = True,
                                                           validation_split=0.1)
      
        k_fold_validation(model = pruned_model, 
                        x = X_test, 
                        y = y_test, 
                        dxy = dxy_test, 
                        folds =cv_folds,
                        metric_type = "MAE")    
        k_fold_validation(model = pruned_model, 
                        x = X_test, 
                        y = y_test, 
                        dxy = dxy_test, 
                        folds =cv_folds,
                        metric_type = "RMSE")   

        __generate_delta_plots__(model = pruned_model,
                                x = X_test,
                                y = y_test,
                                dxy = dxy_test,
                                color = plot_colors[i])
        
        pruned_models.append(pruned_model)
        training_history.append(history)

        # training ends
        i += 1
        init_sparsity += pruning_fraction_step
    
    return pruned_models, training_history

In [ ]:
# pruned_models, training_history = run_iterative_pruning(
#                                                         baseline_model = baseline,
#                                                         target_sparsity = 0.7,
#                                                         training_params = ft_params)
pruned_models, training_history = run_iterative_pruning_v2(x_train_displ, y_train_displ, dxy_train_displ,
                                                          x_test_displ, y_test_displ, dxy_test_displ,
                                                          baseline_model = baseline,
                                                          init_sparsity = 0.1,
                                                          target_sparsity = 0.8,
                                                          pruning_fraction_step = 0.1,
                                                          training_params = ft_params,
                                                          cv_folds = 1)


### Part- V: Saving the Trained Model

In [ ]:
for i in range(len(pruned_models)):
    with open("./trainingLog" + str((i+1)*10), 'wb') as file_pi:
          pickle.dump(training_history[i].history, file_pi)
    model_filename = "custom_model_" + str((i+1)*10)
    saving_pruned_model(model = pruned_models[i], 
                      filepath= sys.path[-1] + "/models", 
                      model_filename = model_filename)